In [21]:
# import the required libraries
import pandas as pd
import numpy as np
import pickle
from datacleaningfuncs import *
import networkx as nx

In [22]:
# import database of structured information from books and the web
database = pd.read_pickle('databasefrom1.pickle')
text = pd.read_pickle('databasetextcomponentfrom1.pickle')
# data containing the document id's and the weights associated with each of them
# documents = pd.read_pickle('documents.pickle')

In [23]:
# initializing Graph of keywords
# gKWs = nx.DiGraph()
gKWs = nx.Graph()

In [24]:
# findingkeywords
everyheadingimportance = 1
everytextimportance = 1
for heading in database['preprocessed_heading']:
    for word in heading:
        if word not in gKWs.nodes():
            gKWs.add_node(word, importance=everyheadingimportance)
        else:
            gKWs.nodes[word]['importance']+=everyheadingimportance
for paragraph in text['preprocessed_lines']:
    for line in paragraph:
        for word in line:
            if word in gKWs.nodes():
                gKWs.nodes[word]['importance']+=everytextimportance

In [25]:
# adding edges between nodes
everyheadingweight = 1
everytextweight = 1
for heading in database['preprocessed_heading']:
    if len(heading)>1:
        for i in range(len(heading)-1):
            one = heading[i]
            two = heading[i+1]
            if (one, two) not in gKWs.edges():
                gKWs.add_edge(one, two, weight=everyheadingweight)
            else:
                gKWs.edges[(one, two)]['weight']+=everyheadingweight
for paragraph in text['preprocessed_lines']:
    for line in paragraph:
        if len(line)>1:
            for i in range(len(line)-1):
                one = line[i]
                two = line[i+1]
                if (one, two) in gKWs.edges():
                    gKWs.edges[(one, two)]['weight']+=everytextweight

In [26]:
# removing hapaxes (both nodes and edges)
nodestoremove = []
for node in gKWs.nodes():
    if gKWs.nodes[node]['importance']==everyheadingimportance:
        nodestoremove.append(node)
gKWs.remove_nodes_from(nodestoremove)
edgestoremove = []
for edge in gKWs.edges():
    if gKWs.edges[edge]['weight']==everyheadingweight:
        edgestoremove.append(edge)
gKWs.remove_edges_from(edgestoremove)

In [39]:
keywords = {}

In [40]:
threshold = 0.5
for node in gKWs.nodes():
    nodeimportance = gKWs.nodes[node]['importance']
    sumofweights = 0
    if len(gKWs.edges(node))==0:
        continue
    for edge in gKWs.edges(node):
        sumofweights+=gKWs.edges[edge]['weight']
    if ((nodeimportance-sumofweights)/nodeimportance)>threshold:
        keywords[node]=True

In [47]:
gKWs.nodes['blockchain']

{'importance': 1692}

In [8]:
# exporting the keyword dictionary as dictionaryofkeywords.pickle
# with open('.pickle', 'wb') as file:
#     pickle.dump(dictionaryofkeywords, file)